In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../../input/data/Casen2022.csv', low_memory=False)
data

,Unnamed: 0,id_vivienda,folio,id_persona,region,area,cod_upm,nse,estrato,hogar,...,men18c,may60c,tipohogar,tot_hog,ind_hacina,indsan,ten_viv,ten_viv_f,allega_ext,allega_int
0,1,1000901,100090101,1,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
1,2,1000901,100090101,2,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
2,3,1000901,100090101,3,16,2,10009,4,1630324,1,...,0,1,3,1,1,1,1,1,0,0
3,4,1000902,100090201,1,16,2,10009,4,1630324,1,...,1,1,5,1,1,2,1,1,0,1
4,5,1000902,100090201,2,16,2,10009,4,1630324,1,...,1,1,5,1,1,2,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202226,202227,4585804,458580401,2,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202227,202228,4585804,458580401,3,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202228,202229,4585807,458580701,1,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0
202229,202230,4585807,458580701,2,9,1,45858,3,910113,1,...,1,0,3,1,1,1,1,1,0,0


In [3]:
datos_proc = data.copy()

In [4]:
# Exploración de variables I
print(data.shape, "\n\n")
print(data.columns, "\n\n")
print(data.dtypes, "\n\n")
print(data['sexo'].value_counts(), "\n\n")
print(data['region'].value_counts(), "\n\n")
print(data['edad'].value_counts(), "\n\n")
print(data['edad'].describe(), "\n\n")

(202231, 919) 


Index(['Unnamed: 0', 'id_vivienda', 'folio', 'id_persona', 'region', 'area',
       'cod_upm', 'nse', 'estrato', 'hogar',
       ...
       'men18c', 'may60c', 'tipohogar', 'tot_hog', 'ind_hacina', 'indsan',
       'ten_viv', 'ten_viv_f', 'allega_ext', 'allega_int'],
      dtype='object', length=919) 


Unnamed: 0     int64
id_vivienda    int64
folio          int64
id_persona     int64
region         int64
               ...  
indsan         int64
ten_viv        int64
ten_viv_f      int64
allega_ext     int64
allega_int     int64
Length: 919, dtype: object 


sexo
2    106575
1     95656
Name: count, dtype: int64 


region
13    38674
5     20552
8     19914
6     13864
7     13739
9     13577
10    10707
14    10603
2      9029
3      9011
16     8706
1      8691
4      8236
15     8152
12     5029
11     3747
Name: count, dtype: int64 


edad
30     3184
22     3101
60     3056
32     3030
50     2993
       ... 
103       5
107       2
105       2
104       2
120   

In [5]:
# Exploración de variables II
print(data['o12'].value_counts().sort_index(), '\n\n')
print(data['o15'].value_counts().sort_index(), '\n\n')
print(data['o18'].value_counts().sort_index(), '\n\n')
print(data['o20'].value_counts().sort_index(), '\n\n')
print(data['o21'].value_counts().sort_index(), '\n\n')

o12
-99.0       25
-88.0      233
 1.0     64075
 2.0      8387
 3.0      8521
 4.0       513
 5.0      3956
Name: count, dtype: int64 


o15
1.0     3234
2.0    20736
3.0     8496
4.0     3367
5.0    46695
6.0      166
7.0     2020
8.0      619
9.0      377
Name: count, dtype: int64 


o18
-99.0      472
-88.0     1891
 1.0     41140
 2.0     11124
 3.0      6736
Name: count, dtype: int64 


o20
-99.0       53
-88.0      258
 1.0     51585
 2.0      6446
 3.0      1944
 4.0      1077
Name: count, dtype: int64 


o21
-99.0       32
-88.0      127
 1.0     48751
 2.0      1188
 3.0     11265
Name: count, dtype: int64 




In [6]:
# Procesamiento
datos_proc = (
    datos_proc
    .assign(
        sexo=lambda x: x['sexo'].map({1: "Hombre", 2: "Mujer"}),
            
        edad_tr=lambda x: np.select([x['edad'].between(18, 39), 
                                         x['edad'].between(40, 64), x['edad'] >= 65],
                                        ["Jovenes", "Adultos", "Adultos mayores"],
                                        default=np.nan),
        
        sexo_edad=lambda x: np.select([(x['sexo'] == "Mujer") & (x['edad_tr'] == "Jovenes"),
                                       (x['sexo'] == "Mujer") & (x['edad_tr'] == "Adultos"),
                                       (x['sexo'] == "Mujer") & (x['edad_tr'] == "Adultos mayores"),
                                       (x['sexo'] == "Hombre") & (x['edad_tr'] == "Jovenes"),
                                       (x['sexo'] == "Hombre") & (x['edad_tr'] == "Adultos"),
                                       (x['sexo'] == "Hombre") & (x['edad_tr'] == "Adultos mayores")],
                                      ["Mujer joven", "Mujer adulta", "Mujer mayor", "Hombre joven", "Hombre adulto", "Hombre mayor"],
                                       default=np.nan),
        
        e6a_no_asiste=lambda x: np.select([x['e6a_no_asiste'].between(6, 7),
                                          x['e6a_no_asiste'].between(8, 11),
                                          x['e6a_no_asiste'] == 12,
                                          x['e6a_no_asiste'] == 13,
                                          x['e6a_no_asiste'].between(14, 15)],
                                         ["Basica", "Media", "Tecnico", "Universitaria", "Postgrado"],
                                          default="Otro"),
        
        educa=lambda x: np.select([x['e6a_no_asiste'] == "Otro",
                                  x['e6a_no_asiste'] == "Basica",
                                  x['e6a_no_asiste'] == "Media",
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Tecnico") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Universitaria") & (x['e6c_completo'] == 2),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 1),
                                  (x['e6a_no_asiste'] == "Postgrado") & (x['e6c_completo'] == 2)],
                                 ["Otro nivel educativo", "Educación Basica", "Educación Media", 
                                  "Técnica completa", "Técnica incompleta", "Universitaria completa", 
                                  "Universitaria incompleta", "Postgrado", "Universitaria completa"]),
        
        region=lambda x: x['region'].map({1: "Región de Tarapacá",
                                         2: "Región de Antofagasta",
                                         3: "Región de Atacama",
                                         4: "Región de Coquimbo",
                                         5: "Región de Valparaíso",
                                         6: "Región de O'Higgins",
                                         7: "Región del Maule",
                                         8: "Región del Biobío",
                                         9: "Región de La Araucanía",
                                         10: "Región de Los Lagos",
                                         11: "Región de Aysén",
                                         12: "Región de Magallanes y Antártica",
                                         13: "Región Metropolitana",
                                         14: "Región de Los Ríos",
                                         15: "Región de Arica y Parinacota",
                                         16: "Región de Ñuble"}),
        
        o12=lambda x:x['o12'].map({
            1: 'Permanente',
            2: 'De temporada o estacional',
            3: 'Ocasional o eventual',
            4: 'A prueba',
            5: 'Por plazo o tiempo determinado',
            -88: np.nan,
            -99: np.nan,
        }),
        
        o15=lambda x:x['o15'].map({
            1: 'Patrón(a) o empleador(a)',
            2: 'Trabajador(a) por cuenta propia',
            3: 'Empleado(a) u obrero(a) del sector público (Gobierno Central o Municipal)',
            4: 'Empleado(a) u obrero(a) de empresas públicas',
            5: 'Empleado(a) u obrero(a) del sector privado',
            6: 'Servicio doméstico puertas adentro',
            7: 'Servicio doméstico puertas afuera',
            8: 'FF.AA. y del Orden',
            9: 'Familiar no remunerado',
        }),
        
        o18=lambda x:x['o18'].map({
            1: 'Plazo indefinido',
            2: 'Plazo fijo',
            3: 'Por obra o faena',
            -88: np.nan,
            -99: np.nan,
        }),
        
        o20=lambda x:x['o20'].map({
            1: 'Jornada completa',
            2: 'Jornada parcial',
            3: 'Jornada prolongada', # jornada extendida respecto al horario de acuerdo laboral
            4: 'Otra',
            -88: np.nan,
            -99: np.nan,
        }),
        
        o21=lambda x:x['o21'].map({
            1: 'Solo diurno',
            2: 'Solo nocturno',
            3: 'Rotativo o turnos',
            -88: np.nan,
            -99: np.nan,
        }))
    
    .rename(columns={'y1': 'ingreso'})
    .dropna(subset=['ingreso'])
    .query('edad>=18 & ingreso>=0 & (o1 == 1 | o2 == 1 | o3 == 1)'))

In [7]:
datos_proc = datos_proc[['sexo', 
                         'edad', 
                         'edad_tr', 
                         'sexo_edad', 
                         'educa', 
                         'region', 
                         'ingreso',
                         'o1',
                         'o2',
                         'o3',
                         'o10',
                         'o12',
                         'o15',
                         'o18',
                         'o20',
                         'o21',
                         ]]

datos_proc.rename(columns={
    'o10': 'hrs_sem',
    'o12': 'temporada_trabajo',
    'o15': 'rol_laboral',
    'o18': 'tipoContrato',
    'o20': 'jornadaLaboral',
    'o21': 'tipoHorario',
}, inplace=True)

In [8]:
# Revisión de variables I
print(datos_proc.shape, "\n\n")
print(datos_proc.columns, "\n\n")
print(datos_proc.dtypes, "\n\n")
print(datos_proc['ingreso'].value_counts(), "\n\n")
print(datos_proc['ingreso'].describe(), "\n\n")
print(datos_proc['sexo'].value_counts(), "\n\n")
print(datos_proc['edad_tr'].value_counts(), "\n\n")
print(datos_proc['sexo_edad'].value_counts(), "\n\n")
print(datos_proc['educa'].value_counts(), "\n\n")
print(datos_proc['region'].value_counts(), "\n\n")

(59644, 16) 


Index(['sexo', 'edad', 'edad_tr', 'sexo_edad', 'educa', 'region', 'ingreso',
       'o1', 'o2', 'o3', 'hrs_sem', 'temporada_trabajo', 'rol_laboral',
       'tipoContrato', 'jornadaLaboral', 'tipoHorario'],
      dtype='object') 


sexo                  object
edad                   int64
edad_tr               object
sexo_edad             object
educa                 object
region                object
ingreso              float64
o1                   float64
o2                   float64
o3                   float64
hrs_sem              float64
temporada_trabajo     object
rol_laboral           object
tipoContrato          object
jornadaLaboral        object
tipoHorario           object
dtype: object 


ingreso
400000.0     9600
500000.0     4849
600000.0     3541
450000.0     3130
800000.0     2344
             ... 
124000.0        1
217448.0        1
1238000.0       1
926000.0        1
1695000.0       1
Name: count, Length: 1185, dtype: int64 


count    5.964400e+04
me

In [9]:
# Revisión de variables II
print(datos_proc['hrs_sem'].describe(), '\n\n')
print(datos_proc['temporada_trabajo'].value_counts().sort_index(), '\n\n')
print(datos_proc['rol_laboral'].value_counts().sort_index(), '\n\n')
print(datos_proc['tipoContrato'].value_counts().sort_index(), '\n\n')
print(datos_proc['jornadaLaboral'].value_counts().sort_index(), '\n\n')
print(datos_proc['tipoHorario'].value_counts().sort_index(), '\n\n')

count    59644.000000
mean        42.201529
std         14.531584
min        -88.000000
25%         44.000000
50%         45.000000
75%         45.000000
max        105.000000
Name: hrs_sem, dtype: float64 


temporada_trabajo
A prueba                            444
De temporada o estacional          5175
Ocasional o eventual               2557
Permanente                        47912
Por plazo o tiempo determinado     3426
Name: count, dtype: int64 


rol_laboral
Empleado(a) u obrero(a) de empresas públicas                                  3233
Empleado(a) u obrero(a) del sector privado                                   45397
Empleado(a) u obrero(a) del sector público (Gobierno Central o Municipal)     8256
FF.AA. y del Orden                                                             603
Servicio doméstico puertas adentro                                             160
Servicio doméstico puertas afuera                                             1995
Name: count, dtype: int64 


tipoC

In [10]:
# Guardar datos
datos_proc.to_csv("../../output/data/Casen2022_proc.csv", index=False)